# Resting state fMRI preprocessing
This notebook contains preprocessing tailored precision mapping in children. 

### 1. Unwarping 
* Measure distortion with the fieldmap
* Eliminate variance (motion, deformations,interactions)

### 2. General fMRI processing
* Slice-time correction
* Rigid realignment (also extract DVARS, FD, motion params)
* Co-registration to the sMRI (T1-weighted structural MRI resampled to 2x2x2mm voxels)

### 3. Resting state processing
* Derive temporal mask based on FD and DVARS
* De-noise to remove:
    - Noise associated with white matter and CSF- delete the GM and smooth what is left
    - Noise associated with background signal - delete brain and smooth what's left
    - Global signal
    - motion regressors
    - Motion derivatives (lagged 8 times)
    - Motion spikes (FD>0.2mm, DVARS>2 SDs from mean)
* Bandpass filter
* delete high motion timepoints and concatenate runs

In [1]:
#import packages
import graphviz
from os import listdir, makedirs
from os.path import isdir
from nipype.interfaces.io import DataSink, SelectFiles, DataGrabber, FreeSurferSource # Data i/o
from nipype.interfaces.utility import IdentityInterface, Function     # utility
from nipype.pipeline.engine import Node, Workflow, MapNode, JoinNode        # pypeline engine
from nipype.interfaces.nipy.preprocess import Trim
from nipype.interfaces.ants import N4BiasFieldCorrection
from nipype.interfaces.fsl import SliceTimer, MCFLIRT, FLIRT, BET, Merge, ExtractROI, TOPUP, ApplyTOPUP
from nipype.interfaces.fsl.utils import Reorient2Std, MotionOutliers
from nipype.interfaces.fsl.maths import ApplyMask, MeanImage
from nipype.interfaces.freesurfer import Resample, Binarize, BBRegister, MRIConvert
from nipype.algorithms.confounds import CompCor
from nipype.interfaces.afni.preprocess import Bandpass
from nipype.interfaces.afni.utils import AFNItoNIFTI
from pandas import DataFrame, read_csv

#set output file type for FSL to NIFTI_GZ
from nipype.interfaces.fsl.preprocess import FSLCommand
FSLCommand.set_default_output_type('NIFTI_GZ')

# MATLAB setup - Specify path to current SPM and the MATLAB's default mode
from nipype.interfaces.matlab import MatlabCommand
MatlabCommand.set_default_paths('~/spm12/toolbox')
MatlabCommand.set_default_matlab_cmd("matlab -nodesktop -nosplash")

# Set study variables
study_home = '/Users/SEAlab/Documents/PPM'
raw_data =  '/Users/SEAlab/Documents/PPM/Data'
output_dir = study_home + '/fMRIproc/preprocessing'

workflow_dir = study_home + '/Workflows'
custom_timings = study_home + '/Misc/slice_timing.txt'
phase_encoding_file = study_home + '/Misc/SEA_rest_encoding.txt'
subject_ids = ['1196_PP3', '1196_PP4']
#subject_ids = ['1196_PP1', '1196_PP2','1196_Preg1', '1196_Preg2', '1196_Preg3', '1196_Preg4', '1196_Preg5']
#subject_ids = ['1196_Preg1', '1196_Preg2', '1196_Preg3', '1196_Preg4', '1196_Preg5']

#reduce to 2 if other people are using the Mac

proc_cores = 4 # number of cores of processing for the workflows

interleave = True
TR = 2 # in seconds
slice_dir = 3 # 1=x, 2=y, 3=z
resampled_voxel_size = (2,2,2)

fd_threshold = 0.2 #in mm
dvars_threshold = 2 # in standard units

highpass_freq = 0.008 #in Hz
lowpass_freq = 0.09 #in Hz

In [2]:
## File handling Nodes

# Identity node for each subject
subinfosource = Node(IdentityInterface(fields=['subject_id']),
                     name='subinfosource')
subinfosource.iterables = [('subject_id', subject_ids)]

# Datasink- where our select outputs will go
substitutions = [('_subject_id_', '')]
datasink = Node(DataSink(), name='datasink')
datasink.inputs.base_directory = output_dir
datasink.inputs.container = output_dir
datasink.inputs.substitutions = substitutions

# Unwarping 


In [3]:
def sort_pes(pes):
    from nipype import config, logging
    from nipype.interfaces.fsl import Merge
    from os.path import abspath
    config.enable_debug_mode()
    logging.update_logging(config)
    
    print(pes)
    pe1s = []
    pe0s = []
    for file in pes:
        if '300d.01' in file:
            pe0s.append(file)
        elif '300d_revPE' in file:
            pe1s.append(file)
    
    if len(pe1s) < len(pe0s):
        for a in range(len(pe1s),len(pe0s)):
            pe1s.append(pe1s[0])
    
    pe1s = sorted(pe1s)
    pe0s = sorted(pe0s)

    me = Merge()
    merged_pes = []
    
    for i in range(0,len(pe1s)):
        num=pe1s[i][-12:-11]
        me.inputs.in_files = [pe1s[i],pe0s[i]]
        me.inputs.dimension='t'
        me.inputs.merged_file = 'merged_pes%s.nii.gz' % num
        me.run()
        file = abspath('merged_pes%s.nii.gz' % num)
        merged_pes.append(file)
        
    return(merged_pes)

In [4]:
pes_template = {'pes': raw_data + '/%s/NIFTI/FMRI_rest*.01.nii'}
select_pes = Node(DataGrabber(sort_filelist=True,
                              template = raw_data + '/%s/NIFTI/FMRI_rest*.01.nii',
                              field_template = pes_template,
                              base_directory=raw_data,
                              infields=['subject_id'], 
                              template_args={'pes':[['subject_id']]}), 
                  name='select_pes')

func_template = {'func': raw_data + '/%s/NIFTI/FMRI_rest*_300d.01.nii'}
select_func = Node(DataGrabber(sort_filelist=True,
                               template = raw_data + '/%s/NIFTI/FMRI_rest*_300d.01.nii',
                               field_template = func_template,
                               base_directory=raw_data,
                               infields=['subject_id'], 
                               template_args={'func':[['subject_id']]}), 
                   name='select_func')

# include only the first volume of each PE volume
trim_PEs = MapNode(ExtractROI(t_min=0, t_size=5),name='trim_PEs', 
                   iterfield=['in_file'])

sort_pe_list = Node(Function(input_names=['pes'],
                             output_names=['merged_pes'],
                             function=sort_pes), 
                    name='sort_pe_list')

topup = MapNode(TOPUP(encoding_file=phase_encoding_file), name='topup',iterfield=['in_file'])

apply_topup = MapNode(ApplyTOPUP(in_index=[6], encoding_file=phase_encoding_file,
                                 method='jac', out_corrected='func_unwarped.nii.gz'),
                      name='apply_topup',iterfield=['in_topup_fieldcoef','in_topup_movpar','in_files'])

In [8]:
prepreprocflow = Workflow(name='unwarpflow')
prepreprocflow.connect([(subinfosource,select_pes, [('subject_id','subject_id')]),
                        (subinfosource,select_func, [('subject_id','subject_id')]),
                        (select_pes,trim_PEs, [('pes','in_file')]), 
                        (trim_PEs,sort_pe_list, [('roi_file','pes')]),
                        (sort_pe_list,topup, [('merged_pes','in_file')]),
                        (topup, apply_topup, [('out_fieldcoef','in_topup_fieldcoef'), 
                                              ('out_movpar','in_topup_movpar')]),
                        (select_func, apply_topup, [('func','in_files')]),
                        (apply_topup, datasink, [('out_corrected','unwarped_funcs')])
                       ])

prepreprocflow.base_dir = workflow_dir
prepreprocflow.write_graph(graph2use='flat')
prepreprocflow.run('MultiProc', plugin_args={'n_procs': proc_cores, 'memory_gb':8})

210625-14:26:05,427 nipype.workflow INFO:
	 Generated workflow graph: /Users/SEAlab/Documents/PPM/Workflows/unwarpflow/graph.png (graph2use=flat, simple_form=True).
210625-14:26:05,577 nipype.workflow INFO:
	 Workflow unwarpflow settings: ['check', 'execution', 'logging', 'monitoring']
210625-14:26:06,120 nipype.workflow INFO:
	 Running in parallel.
210625-14:26:06,126 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 4 jobs ready. Free memory (GB): 8.00/8.00, Free processors: 4/4.
210625-14:26:06,383 nipype.workflow INFO:
	 [Node] Setting-up "unwarpflow.select_func" in "/Users/SEAlab/Documents/PPM/Workflows/unwarpflow/_subject_id_1196_PP4/select_func".210625-14:26:06,389 nipype.workflow INFO:
	 [Node] Setting-up "unwarpflow.select_func" in "/Users/SEAlab/Documents/PPM/Workflows/unwarpflow/_subject_id_1196_PP3/select_func".210625-14:26:06,386 nipype.workflow INFO:
	 [Node] Setting-up "unwarpflow.select_pes" in "/Users/SEAlab/Documents/PPM/Workflows/unwarpflow/_subject_id_1196_PP

210625-14:26:28,677 nipype.interface DEBUG:
	 in_files_['/Users/SEAlab/Documents/PPM/Workflows/unwarpflow/_subject_id_1196_PP4/trim_PEs/mapflow/_trim_PEs1/FMRI_rest1_300d_revPE.01_roi.nii.gz', '/Users/SEAlab/Documents/PPM/Workflows/unwarpflow/_subject_id_1196_PP4/trim_PEs/mapflow/_trim_PEs3/FMRI_rest3_300d.01_roi.nii.gz']
210625-14:26:28,692 nipype.interface DEBUG:
	 merged_file_merged_pes1.nii.gz
210625-14:26:28,818 nipype.workflow DEBUG:
	 Needed files: /Users/SEAlab/Documents/PPM/Workflows/unwarpflow/_subject_id_1196_PP3/sort_pe_list/merged_pes1.nii.gz;/Users/SEAlab/Documents/PPM/Workflows/unwarpflow/_subject_id_1196_PP3/sort_pe_list/merged_pes1.nii.gz;/Users/SEAlab/Documents/PPM/Workflows/unwarpflow/_subject_id_1196_PP3/sort_pe_list/_0x8e05185c518a478221ac502070be4ae1_unfinished.json;/Users/SEAlab/Documents/PPM/Workflows/unwarpflow/_subject_id_1196_PP3/sort_pe_list/_inputs.pklz;/Users/SEAlab/Documents/PPM/Workflows/unwarpflow/_subject_id_1196_PP3/sort_pe_list/_node.pklz
210625-14:2

	 out_field_merged_pes1_field.nii.gz

210625-14:26:34,252 nipype.interface DEBUG:
	 out_jac_prefix_jac210625-14:26:34,252 nipype.interface DEBUG:
	 out_field_merged_pes1_field.nii.gz

210625-14:26:34,255 nipype.interface DEBUG:
	 out_jac_prefix_jac210625-14:26:34,255 nipype.interface DEBUG:
	 out_logfile_merged_pes1_topup.log

210625-14:26:34,259 nipype.interface DEBUG:
	 out_mat_prefix_xfm210625-14:26:34,259 nipype.interface DEBUG:
	 out_logfile_merged_pes1_topup.log

210625-14:26:34,263 nipype.interface DEBUG:
	 out_mat_prefix_xfm210625-14:26:34,263 nipype.interface DEBUG:
	 out_warp_prefix_warpfield

210625-14:26:34,267 nipype.interface DEBUG:
	 out_warp_prefix_warpfield210625-14:26:34,267 nipype.workflow INFO:
	 [Node] Running "_topup0" ("nipype.interfaces.fsl.epi.TOPUP"), a CommandLine Interface with command:
topup --config=b02b0.cnf --datain=/Users/SEAlab/Documents/PPM/Misc/SEA_rest_encoding.txt --imain=/Users/SEAlab/Documents/PPM/Workflows/unwarpflow/_subject_id_1196_PP4/sort_pe

	 /Users/SEAlab/Documents/PPM/Workflows/unwarpflow/_subject_id_1196_PP3/apply_topup/mapflow/_apply_topup0/merged_pes1_base_fieldcoef.nii.gz
210625-15:17:13,450 nipype.utils DEBUG:
	 File: /Users/SEAlab/Documents/PPM/Workflows/unwarpflow/_subject_id_1196_PP3/apply_topup/mapflow/_apply_topup0/merged_pes1_base_fieldcoef.nii.gz already exists, not overwriting, copy:0
210625-15:17:13,453 nipype.utils DEBUG:
	 /Users/SEAlab/Documents/PPM/Workflows/unwarpflow/_subject_id_1196_PP3/apply_topup/mapflow/_apply_topup0/merged_pes1_base_movpar.txt
210625-15:17:13,456 nipype.utils DEBUG:
	 Symlinking File: /Users/SEAlab/Documents/PPM/Workflows/unwarpflow/_subject_id_1196_PP3/apply_topup/mapflow/_apply_topup0/merged_pes1_base_movpar.txt->/Users/SEAlab/Documents/PPM/Workflows/unwarpflow/_subject_id_1196_PP3/topup/mapflow/_topup0/merged_pes1_base_movpar.txt
210625-15:17:13,459 nipype.utils DEBUG:
	 /Users/SEAlab/Documents/PPM/Workflows/unwarpflow/_subject_id_1196_PP3/apply_topup/mapflow/_apply_topup0/me

210625-15:17:17,130 nipype.workflow INFO:
	 [Node] Setting-up "_apply_topup1" in "/Users/SEAlab/Documents/PPM/Workflows/unwarpflow/_subject_id_1196_PP3/apply_topup/mapflow/_apply_topup1".
210625-15:17:17,132 nipype.workflow DEBUG:
	 [Node] Not cached "/Users/SEAlab/Documents/PPM/Workflows/unwarpflow/_subject_id_1196_PP3/apply_topup/mapflow/_apply_topup1".
210625-15:17:17,136 nipype.utils DEBUG:
	 Removing contents of /Users/SEAlab/Documents/PPM/Workflows/unwarpflow/_subject_id_1196_PP3/apply_topup/mapflow/_apply_topup1
210625-15:17:17,140 nipype.workflow DEBUG:
	 [Node] Writing pre-exec report to "/Users/SEAlab/Documents/PPM/Workflows/unwarpflow/_subject_id_1196_PP3/apply_topup/mapflow/_apply_topup1/_report/report.rst"
210625-15:17:17,146 nipype.workflow DEBUG:
	 copying files to wd [execute=True, linksonly=False]
210625-15:17:17,151 nipype.utils DEBUG:
	 /Users/SEAlab/Documents/PPM/Workflows/unwarpflow/_subject_id_1196_PP3/apply_topup/mapflow/_apply_topup1/merged_pes1_base_fieldcoef.n

210625-15:18:09,191 nipype.workflow INFO:
	 [Node] Outdated cache found for "unwarpflow.apply_topup".
210625-15:18:09,195 nipype.workflow INFO:
	 [Node] Setting-up "unwarpflow.apply_topup" in "/Users/SEAlab/Documents/PPM/Workflows/unwarpflow/_subject_id_1196_PP3/apply_topup".
210625-15:18:09,198 nipype.workflow DEBUG:
	 adding multipath trait: in_files
210625-15:18:09,201 nipype.workflow DEBUG:
	 adding multipath trait: in_topup_fieldcoef
210625-15:18:09,205 nipype.workflow DEBUG:
	 adding multipath trait: in_topup_movpar
210625-15:18:09,209 nipype.workflow DEBUG:
	 [Node] Hashes: [('encoding_file', ('/Users/SEAlab/Documents/PPM/Misc/SEA_rest_encoding.txt', 'a85cd920feb4e239060b248a12687899')), ('in_files', [[('/Users/SEAlab/Documents/PPM/Data/1196_PP3/NIFTI/FMRI_rest1_300d.01.nii', 'fdc5ee2d6e7bedd887bb41f25b1b33f0')], [('/Users/SEAlab/Documents/PPM/Data/1196_PP3/NIFTI/FMRI_rest2_300d.01.nii', '867154018faf7702c26bab4421309620')]]), ('in_index', [6]), ('in_topup_fieldcoef', [('/Users/

	 sub: /Users/SEAlab/Documents/PPM/fMRIproc/preprocessing/unwarped_funcs/_subject_id_1196_PP3/_apply_topup1/func_unwarped.nii.gz -> /Users/SEAlab/Documents/PPM/fMRIproc/preprocessing/unwarped_funcs/1196_PP3/_apply_topup1/func_unwarped.nii.gz
210625-15:18:23,798 nipype.interface DEBUG:
	 copyfile: /Users/SEAlab/Documents/PPM/Workflows/unwarpflow/_subject_id_1196_PP3/apply_topup/mapflow/_apply_topup1/func_unwarped.nii.gz /Users/SEAlab/Documents/PPM/fMRIproc/preprocessing/unwarped_funcs/1196_PP3/_apply_topup1/func_unwarped.nii.gz
210625-15:18:23,807 nipype.utils DEBUG:
	 /Users/SEAlab/Documents/PPM/fMRIproc/preprocessing/unwarped_funcs/1196_PP3/_apply_topup1/func_unwarped.nii.gz
210625-15:18:23,811 nipype.utils DEBUG:
	 Linking File: /Users/SEAlab/Documents/PPM/fMRIproc/preprocessing/unwarped_funcs/1196_PP3/_apply_topup1/func_unwarped.nii.gz->/Users/SEAlab/Documents/PPM/Workflows/unwarpflow/_subject_id_1196_PP3/apply_topup/mapflow/_apply_topup1/func_unwarped.nii.gz
210625-15:18:23,816 nip

	 Symlinking File: /Users/SEAlab/Documents/PPM/Workflows/unwarpflow/_subject_id_1196_PP4/apply_topup/mapflow/_apply_topup0/merged_pes1_base_fieldcoef.nii.gz->/Users/SEAlab/Documents/PPM/Workflows/unwarpflow/_subject_id_1196_PP4/topup/mapflow/_topup0/merged_pes1_base_fieldcoef.nii.gz210625-15:45:55,841 nipype.utils DEBUG:
	 Symlinking File: /Users/SEAlab/Documents/PPM/Workflows/unwarpflow/_subject_id_1196_PP4/apply_topup/mapflow/_apply_topup3/merged_pes1_base_fieldcoef.nii.gz->/Users/SEAlab/Documents/PPM/Workflows/unwarpflow/_subject_id_1196_PP4/topup/mapflow/_topup3/merged_pes1_base_fieldcoef.nii.gz

210625-15:45:55,852 nipype.utils DEBUG:
	 /Users/SEAlab/Documents/PPM/Workflows/unwarpflow/_subject_id_1196_PP4/apply_topup/mapflow/_apply_topup0/merged_pes1_base_fieldcoef.nii.gz210625-15:45:55,852 nipype.utils DEBUG:
	 /Users/SEAlab/Documents/PPM/Workflows/unwarpflow/_subject_id_1196_PP4/apply_topup/mapflow/_apply_topup3/merged_pes1_base_fieldcoef.nii.gz

210625-15:45:55,855 nipype.utils

## Process T1 anat for fMRI processing
1. Resample skullstripped T1w anat
2. Resample segmentation 
3. Create tissue masks:
    - GM only
    - Non-GM only
    - Full brain (no ventricles)
    - Full brain (with ventricles and dilated)
    - Extra-cerebral space
4. Resample surfaces

In [5]:
def make_masks_freesurfer(aseg_nifti):
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    from nibabel import load, save, Nifti1Image
    from numpy import zeros
    from nipype.interfaces.freesurfer import Binarize
    from os.path import abspath
    
    # load the subject's segmentation volumes
    aseg_img = load(aseg_nifti)
    aseg_data = aseg_img.get_data()
    
    # list indices for each tissue type (from freesurfer aseg)
    wm = [2, 7, 41, 46, 60, 28, 16, 77, 251, 252, 253, 254, 255]
    gm = [3, 6, 8, 9, 10, 11, 12, 13, 17, 18, 26, 42, 47, 49, 50, 51, 52, 53, 54, 58]
    csf = [4, 5, 14, 15, 24, 63]
    non_gm = wm + csf
    whole_brain = wm + gm + csf
    
    # preallocate zeros for the tissue masks
    wm_combined = zeros(aseg_data.shape)
    gm_combined = zeros(aseg_data.shape)
    csf_combined = zeros(aseg_data.shape)
    non_gm_combined = zeros(aseg_data.shape)
    whole_brain_combined = zeros(aseg_data.shape)
    
    # put labels into lists to make looping easier
    tissues_idx = [wm, gm, csf, non_gm, whole_brain]
    tissues_data = [wm_combined, gm_combined, csf_combined, non_gm_combined, whole_brain_combined]
    tissue_name = ['wm','gm','csf','non_gm','whole_brain']
    
    # initialize the binarize class
    binarize = Binarize()
    binarize.inputs.min = 0.5
    binarize.inputs.dilate = 1
    binarize.inputs.erode = 1
    
    # isolate labels for specific regions
    for x in range(0,5):
        if 'wm' in tissue_name[x] or 'non_gm' in tissue_name[x]:
            binarize.inputs.erode = 2
        else:
            binarize.inputs.erode = 1
        for label in tissues_idx[x]:
            tissues_data[x][aseg_data==label] = 1
        tmp_img = Nifti1Image(tissues_data[x],header=aseg_img.header, affine=aseg_img.affine)
        save(tmp_img, tissue_name[x] + '_mask.nii.gz')
         
        binarize.inputs.in_file = abspath(tissue_name[x] + '_mask.nii.gz')
        binarize.inputs.binary_file = tissue_name[x] + '_mask.nii.gz'
        binarize.run()
    
    # make dilated brain mask
    binarize.inputs.in_file = abspath('whole_brain_mask.nii.gz')
    binarize.inputs.binary_file = 'whole_brain_D1_mask.nii.gz'
    binarize.inputs.dilate = 1
    binarize.inputs.erode = 0
    binarize.run()
    
    # gather up the mask filepaths to return
    wm_csf_mask = abspath('non_gm_mask.nii.gz')
    brain_mask = abspath('whole_brain_mask.nii.gz')
    gm_only_mask = abspath('gm_mask.nii.gz')
    brain_mask_dilated = abspath('whole_brain_D1_mask.nii.gz')
    
    return(wm_csf_mask, brain_mask, gm_only_mask, brain_mask_dilated)

def invert_masks(mask_file):
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    from nibabel import load, save, Nifti1Image
    from os.path import abspath, basename
    from numpy import zeros
    
    # load mask data
    mask_img = load(mask_file)
    mask_data = mask_img.get_data()
    
    # preallocate new mask data
    inv_data = zeros(mask_data.shape)
    
    # invert the 1s and 0s
    inv_data[mask_data==0] = 1
    
    #save as a new file
    inv_img = Nifti1Image(inv_data, header = mask_img.header, affine = mask_img.affine)
    save(inv_img, 'inverted_' + basename(mask_file))
    inverted_mask = abspath('inverted_' + basename(mask_file))
    
    return(inverted_mask)

#Function get FS Dir
def get_subject_fs_dir(datadir, subject_id):
    fs_dir = datadir + "/" + subject_id + "/T1w"
    return(fs_dir)


In [6]:
#Choose FS Directory 
get_fsdir = Node(Function(input_names=['datadir', 'subject_id'], 
                                        output_names=['fs_dir'],
                         function=get_subject_fs_dir), name='get_fsdir')
get_fsdir.inputs.datadir=raw_data

# FreeSurferSource node to grab processed T1w data from freesurfer
t1w_source = Node(FreeSurferSource(), name='t1w_source')

# resample brain file
resample_anat = Node(MRIConvert(vox_size=resampled_voxel_size, 
                                out_type='niigz', out_file='resampled_anat.nii.gz'), name='resample_anat')

# resample segmentation file
resample_seg = Node(MRIConvert(vox_size=resampled_voxel_size, 
                               resample_type='nearest', 
                               out_type='niigz'), name='resample_seg')

# make masks
make_masks = Node(Function(input_names=['aseg_nifti'], 
                           output_names=['wm_csf_mask', 'brain_mask', 'gm_only_mask', 'brain_mask_dilated'], 
                           function=make_masks_freesurfer), name='make_masks')

#invert the brain mask
invert_mask = Node(Function(input_names=['mask_file'], 
                            output_names=['inverted_mask'], 
                            function=invert_masks), name='invert_mask')

In [11]:
anatpreproc = Workflow(name='fmri_anat_preprocflow')
anatpreproc.connect([(subinfosource, t1w_source, [('subject_id','subject_id')]),
                     (subinfosource, get_fsdir, [('subject_id', 'subject_id')]),
                     (get_fsdir, t1w_source, [('fs_dir', 'subjects_dir')]),
                     (t1w_source, resample_anat, [('brain','in_file')]),
                     (t1w_source, resample_seg, [('aseg','in_file')]),
                     (resample_seg, make_masks, [('out_file','aseg_nifti')]),
                     (make_masks, invert_mask, [('brain_mask','mask_file')]),
                     
                     (make_masks, datasink, [('wm_csf_mask','wm_csf_mask'),
                                             ('brain_mask','brain_mask'),
                                             ('gm_only_mask','gm_mask'),
                                             ('brain_mask_dilated','brain_mask_dilated')]),
                     (resample_anat, datasink, [('out_file','resampled_t1w_anat')]),
                     (invert_mask, datasink, [('inverted_mask','background_mask')])
                    ])
anatpreproc.base_dir = workflow_dir
#anatpreproc.write_graph(graph2use='flat')
anatpreproc.run('MultiProc', plugin_args={'n_procs': proc_cores})

210625-15:48:59,110 nipype.workflow INFO:
	 Workflow fmri_anat_preprocflow settings: ['check', 'execution', 'logging', 'monitoring']
210625-15:49:01,110 nipype.workflow INFO:
	 Running in parallel.
210625-15:49:01,290 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 2 jobs ready. Free memory (GB): 14.40/14.40, Free processors: 4/4.
210625-15:49:01,372 nipype.workflow INFO:
	 [Job 0] Cached (fmri_anat_preprocflow.get_fsdir).
210625-15:49:01,375 nipype.workflow INFO:
	 [Job 7] Cached (fmri_anat_preprocflow.get_fsdir).
210625-15:49:03,628 nipype.workflow INFO:
	 [Node] Setting-up "fmri_anat_preprocflow.t1w_source" in "/Users/SEAlab/Documents/PPM/Workflows/fmri_anat_preprocflow/_subject_id_1196_PP4/t1w_source".
210625-15:49:03,642 nipype.workflow INFO:
	 [Node] Running "t1w_source" ("nipype.interfaces.io.FreeSurferSource")
210625-15:49:03,972 nipype.workflow INFO:
	 [Node] Setting-up "fmri_anat_preprocflow.t1w_source" in "/Users/SEAlab/Documents/PPM/Workflows/fmri_anat_preprocflow/

## Preprocess fMRI resting state data
These nodes and workflow (preprocflow) perform basic preprocessing to align the functional volumes into a common space.
1. Reorient images to standard space
2. Reslice the structural image to 2mm isotropic
3. Functional image slice time correction
4. Rigid realignment to middle volume of functional image
5. Coregistration of functional images to structural image

In [7]:
## File handling Nodes
substitutions = [('_subject_id_', '_'),('_filename_','')]
datasink.inputs.substitutions = substitutions

# Select anat
anat_template = {'brain_mask': output_dir + '/brain_mask_dilated/{subject_id}/whole_brain_D1_mask.nii.gz',
                 'resliced_anat': output_dir + '/resampled_t1w_anat/{subject_id}/resampled_anat.nii.gz'}
selectanat = Node(SelectFiles(anat_template), name='selectanat')

# Data grabber- select fMRI
func_template = {'func':output_dir + '/unwarped_funcs/{subject_id}/_apply_topup{run_num}/func_unwarped.nii.gz'}
selectfunc = Node(SelectFiles(func_template), name='selectfunc')
selectfunc.iterables=('run_num', ['0','1','2','3'])

In [8]:
def list_func(template,subject_id,session_info):
    func = []
    filelist=session_info[session_info['subject_id']==int(subject_id)]['run_name'].to_list()
    for f in filelist:
        func.append(template.format(subject_id, f))
        
    print(func)
    return(func)

def norm_timeseries(in_file, mask_file):
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    from os.path import abspath
    from nilearn.masking import apply_mask, unmask
    from sklearn.preprocessing import StandardScaler
    
    raw_data = apply_mask(in_file, mask_file)
    scaler = StandardScaler(with_mean=True)
    norm_data = scaler.fit_transform(raw_data)
    norm_data = norm_data + 1000
    img = unmask(norm_data, mask_file)
    img.to_filename('norm_func.nii.gz')
    out_file = abspath('norm_func.nii.gz')
    return(out_file)

#Function get FS Dir
def get_subject_fs_dir(datadir, subject_id):
    fs_dir = datadir + "/" + subject_id + "/T1w"
    return(fs_dir)

In [9]:
## Nodes for preprocessing

#Slice timing correction based on interleaved acquisition using FSL
slicetime_correct = Node(SliceTimer(interleaved=interleave, 
                                    time_repetition=TR, 
                                    out_file='st_func.nii.gz'),
                         name='slicetime_correct')
# Rigid realignment
realign = Node(MCFLIRT(out_file='rest_moco.nii.gz',save_plots=True), 
               name='realign')

# compute DVARS and FD
calc_dvars = Node(MotionOutliers(threshold=0.5, out_metric_values='out_mot_metric.txt',
                                 out_metric_plot='plot.png', metric='dvars'),
                     name='calc_dvars')

calc_fd = Node(MotionOutliers(threshold=0.01, out_metric_values='out_mot_metric.txt',
                              out_metric_plot='plot.png', metric='fd'),
               name='calc_fd')

# Registration- using bbregister
coreg = Node(BBRegister(contrast_type='bold', out_fsl_file=True,
                        registered_file='warped_func.nii.gz'), 
             name='coreg', iterfield=['source_file'])

apply_registration = Node(FLIRT(apply_xfm=True, out_file='reg_func.nii.gz'), 
                          name='apply_registration')

# Resample functional
resample_func = Node(MRIConvert(vox_size=resampled_voxel_size, 
                                out_type='niigz', out_file='func.nii.gz'), 
                     name='resample_func', iterfield=['in_file'])

# normalize data within each run
normalize = Node(Function(input_names=['in_file','mask_file'], 
                          output_names=['out_file'], 
                          function=norm_timeseries), 
                 name='normalize')

In [10]:
## Preprocessing Workflow
fmripreproc = Workflow(name='fmri_preprocflow')
fmripreproc.connect([(subinfosource,selectanat,[('subject_id','subject_id')]), 
                     (subinfosource,selectfunc,[('subject_id','subject_id')]), 
                     (subinfosource,coreg,[('subject_id','subject_id')]), 
                     (subinfosource, get_fsdir,[('subject_id', 'subject_id')]),
                     (get_fsdir, coreg,[('fs_dir','subjects_dir')]),
                     (selectfunc,slicetime_correct,[('func','in_file')]),
                     (slicetime_correct,realign,[('slice_time_corrected_file','in_file')]),
                     (slicetime_correct,calc_dvars,[('slice_time_corrected_file','in_file')]),
                     (slicetime_correct,calc_fd,[('slice_time_corrected_file','in_file')]),
                     (realign,apply_registration,[('out_file','in_file')]),
                     (realign,coreg, [('out_file','source_file')]),
                     (selectanat, apply_registration, [('resliced_anat','reference')]),
                     (coreg, apply_registration, [('out_fsl_file','in_matrix_file')]),
                     
                     (apply_registration, normalize,[('out_file','in_file')]),
                     (selectanat,normalize,[('brain_mask','mask_file')]),
                   
                     (realign, datasink,[('par_file','motion_parameters')]),
                     (calc_dvars, datasink,[('out_metric_plot','dvars_plot'),
                                            ('out_metric_values','dvars_values')]),
                     (calc_fd, datasink, [('out_metric_plot','fd_plot'),
                                          ('out_metric_values','fd_values')]),
                     (normalize, datasink, [('out_file','registered_func')])
                    ])
fmripreproc.base_dir = workflow_dir
#fmripreproc.write_graph(graph2use='flat')
fmripreproc.run('MultiProc', plugin_args={'n_procs': proc_cores})

210625-18:48:36,728 nipype.workflow INFO:
	 Workflow fmri_preprocflow settings: ['check', 'execution', 'logging', 'monitoring']
210625-18:48:37,265 nipype.workflow INFO:
	 Running in parallel.
210625-18:48:37,272 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 12 jobs ready. Free memory (GB): 14.40/14.40, Free processors: 4/4.
210625-18:48:37,415 nipype.workflow INFO:
	 [Node] Setting-up "fmri_preprocflow.selectfunc" in "/Users/SEAlab/Documents/PPM/Workflows/fmri_preprocflow/_subject_id_1196_PP4/_run_num_3/selectfunc".210625-18:48:37,417 nipype.workflow INFO:
	 [Node] Setting-up "fmri_preprocflow.selectfunc" in "/Users/SEAlab/Documents/PPM/Workflows/fmri_preprocflow/_subject_id_1196_PP4/_run_num_1/selectfunc".210625-18:48:37,418 nipype.workflow INFO:
	 [Node] Setting-up "fmri_preprocflow.selectfunc" in "/Users/SEAlab/Documents/PPM/Workflows/fmri_preprocflow/_subject_id_1196_PP4/_run_num_0/selectfunc".210625-18:48:37,416 nipype.workflow INFO:
	 [Node] Setting-up "fmri_preprocfl

210625-18:48:49,286 nipype.workflow ERROR:
	 Node selectfunc.aI.a2.b0 failed to run on host PSY-189J1G9.local.
210625-18:48:49,287 nipype.workflow ERROR:
	 Saving crash info to /Users/SEAlab/Documents/PPM/connectivity_analysis_PPM/crash-20210625-184849-SEAlab-selectfunc.aI.a2.b0-b88e208f-1a42-41ae-87fc-36219a85a82b.pklz
Traceback (most recent call last):
  File "/Users/SEAlab/anaconda3/lib/python3.6/site-packages/nipype/pipeline/plugins/multiproc.py", line 148, in __init__
    mp_context=mp_context,
TypeError: __init__() got an unexpected keyword argument 'initializer'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/SEAlab/anaconda3/lib/python3.6/site-packages/nipype/pipeline/plugins/multiproc.py", line 67, in run_node
    result["result"] = node.run(updatehash=updatehash)
  File "/Users/SEAlab/anaconda3/lib/python3.6/site-packages/nipype/pipeline/engine/nodes.py", line 516, in run
    result = self._run_interface(

	 sub: /Users/SEAlab/Documents/PPM/fMRIproc/preprocessing/dvars_plot/_subject_id_1196_PP4/_run_num_1/plot.png -> /Users/SEAlab/Documents/PPM/fMRIproc/preprocessing/dvars_plot/_1196_PP4/_run_num_1/plot.png
210625-18:48:56,112 nipype.interface INFO:
	 sub: /Users/SEAlab/Documents/PPM/fMRIproc/preprocessing/dvars_values/_subject_id_1196_PP4/_run_num_1/out_mot_metric.txt -> /Users/SEAlab/Documents/PPM/fMRIproc/preprocessing/dvars_values/_1196_PP4/_run_num_1/out_mot_metric.txt
210625-18:48:56,119 nipype.interface INFO:
	 sub: /Users/SEAlab/Documents/PPM/fMRIproc/preprocessing/fd_plot/_subject_id_1196_PP4/_run_num_1/plot.png -> /Users/SEAlab/Documents/PPM/fMRIproc/preprocessing/fd_plot/_1196_PP4/_run_num_1/plot.png
210625-18:48:56,123 nipype.interface INFO:
	 sub: /Users/SEAlab/Documents/PPM/fMRIproc/preprocessing/fd_values/_subject_id_1196_PP4/_run_num_1/out_mot_metric.txt -> /Users/SEAlab/Documents/PPM/fMRIproc/preprocessing/fd_values/_1196_PP4/_run_num_1/out_mot_metric.txt
210625-18:48:56

210625-18:50:22,992 nipype.workflow INFO:
	 [Node] Finished "fmri_preprocflow.realign".
210625-18:50:23,400 nipype.workflow INFO:
	 [Job 52] Completed (fmri_preprocflow.realign).
210625-18:50:23,402 nipype.workflow INFO:
	 [MultiProc] Running 2 tasks, and 1 jobs ready. Free memory (GB): 14.00/14.40, Free processors: 2/4.
                     Currently running:
                       * fmri_preprocflow.realign
                       * fmri_preprocflow.calc_dvars
210625-18:50:23,445 nipype.workflow INFO:
	 [Node] Setting-up "fmri_preprocflow.coreg" in "/Users/SEAlab/Documents/PPM/Workflows/fmri_preprocflow/_subject_id_1196_PP3/_run_num_1/coreg".
210625-18:50:23,596 nipype.workflow INFO:
	 [Node] Running "coreg" ("nipype.interfaces.freesurfer.preprocess.BBRegister"), a CommandLine Interface with command:
bbregister --bold --fslmat /Users/SEAlab/Documents/PPM/Workflows/fmri_preprocflow/_subject_id_1196_PP3/_run_num_1/coreg/rest_moco_bbreg_1196_PP3.mat --reg /Users/SEAlab/Documents/PPM/Work

210625-19:07:40,738 nipype.workflow INFO:
	 [Node] Setting-up "fmri_preprocflow.normalize" in "/Users/SEAlab/Documents/PPM/Workflows/fmri_preprocflow/_subject_id_1196_PP3/_run_num_1/normalize".
210625-19:07:43,697 nipype.workflow INFO:
	 [Node] Running "normalize" ("nipype.interfaces.utility.wrappers.Function")
210625-19:09:25,131 nipype.workflow DEBUG:
	 Needed files: /Users/SEAlab/Documents/PPM/Workflows/fmri_preprocflow/_subject_id_1196_PP3/_run_num_1/normalize/norm_func.nii.gz;/Users/SEAlab/Documents/PPM/Workflows/fmri_preprocflow/_subject_id_1196_PP3/_run_num_1/normalize/_0xc0ab808eb16b4ccbbca0723650f3284a_unfinished.json;/Users/SEAlab/Documents/PPM/Workflows/fmri_preprocflow/_subject_id_1196_PP3/_run_num_1/normalize/_inputs.pklz;/Users/SEAlab/Documents/PPM/Workflows/fmri_preprocflow/_subject_id_1196_PP3/_run_num_1/normalize/_node.pklz
210625-19:09:25,137 nipype.workflow DEBUG:
	 Needed dirs: /Users/SEAlab/Documents/PPM/Workflows/fmri_preprocflow/_subject_id_1196_PP3/_run_num_1/no

210625-19:09:45,538 nipype.workflow ERROR:
	 could not run node: fmri_preprocflow.selectfunc.aI.a2.b0
210625-19:09:45,539 nipype.workflow INFO:
	 crashfile: /Users/SEAlab/Documents/PPM/connectivity_analysis_PPM/crash-20210625-184849-SEAlab-selectfunc.aI.a2.b0-b88e208f-1a42-41ae-87fc-36219a85a82b.pklz
210625-19:09:45,539 nipype.workflow INFO:
	 ***********************************


RuntimeError: Workflow did not execute cleanly. Check log for details

## Create Nuissance Regressors
These nodes and workflow creates both the subject specific and general nuissance regressors needed for preprocessing the rest data

In [21]:
# Data grabber
selectfiles_template = {'brain_mask':output_dir + '/brain_mask/{subject_id}/whole_brain_mask.nii.gz',
                        'nonbrain_mask': output_dir + '/background_mask/{subject_id}/inverted_whole_brain_mask.nii.gz', 
                        'nongm_mask':output_dir + '/wm_csf_mask/{subject_id}/non_gm_mask.nii.gz', 
                        'func': output_dir + '/registered_func/{subject_id}/_run_num_{run_num}/norm_func.nii.gz', 
                        'motion': output_dir + '/motion_parameters/{subject_id}/_run_num_{run_num}/rest_moco.nii.gz.par',
                        'fd': output_dir + '/fd_values/{subject_id}/_run_num_{run_num}/out_mot_metric.txt',
                        'dvars': output_dir + '/dvars_values/{subject_id}/_run_num_{run_num}/out_mot_metric.txt'}
selectfiles = Node(SelectFiles(selectfiles_template), name='selectmask')
selectfiles.iterables=('run_num', ['0','1','2','3'])

In [22]:
def mask_blur_func(mask, in_file):
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    from os.path import abspath
    from numpy import median, where
    from nipype.interfaces.fsl import ApplyMask
    from glob import glob
    from subprocess import check_call

    applymask = ApplyMask()
    applymask.inputs.mask_file = mask
    applymask.inputs.in_file = in_file
    applymask.inputs.out_file = 'masked_file.nii.gz'
    applymask.inputs.nan2zeros = True
    applymask.run()

    masked_file = abspath('masked_file.nii.gz')
    
    
    if 'nonbrain' in mask:
        check_call(['gunzip',masked_file])
        
        from nipype.interfaces.spm import Smooth
        smooth = Smooth()
        smooth.inputs.in_files = 'masked_file.nii'
        smooth.inputs.fwhm = [22,4,4]
        smooth.inputs.out_prefix = 'blurred_'
        smooth.run()
        check_call(['gzip','blurred_masked_file.nii'])
        
    else:
        from nipype.interfaces.fsl import Smooth
        smooth = Smooth()
        smooth.inputs.in_file = masked_file
        smooth.inputs.smoothed_file = 'blurred_masked_file.nii.gz'
        smooth.inputs.fwhm = 4
        smooth.run()

    blurred_masked_file = abspath('blurred_masked_file.nii.gz')

    return(blurred_masked_file)

def leadlagmatrix(motion_file):
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    from os.path import abspath
    import numpy as np

    motion_params = np.loadtxt(motion_file, dtype=float)
    trs = motion_params.shape[0]
    params = motion_params.shape[1]
    num_lags = 6
    derivatives = np.gradient(motion_params, axis=0)
    leadlagderivs = np.zeros((trs,params*num_lags))
    derivativessq = derivatives**2
    leadlagderivssq = np.zeros((trs,params*num_lags))

    for i in range(0,params):
        for j in range(0,num_lags):
            leadlagderivs[:,j+num_lags*i] =  np.roll(derivatives[:,i],shift=j, axis=0)
            leadlagderivs[:j,j+num_lags*i] = 0

    for i in range(0,params):
        for j in range(0,num_lags):
            leadlagderivssq[:,j+num_lags*i] =  np.roll(derivativessq[:,i],shift=j, axis=0)
            leadlagderivssq[:j,j+num_lags*i] = 0

    np.savetxt('derivsleadlag.txt', leadlagderivs)
    np.savetxt('derivssqleadlag.txt', leadlagderivssq)

    leadlagderivsmot = abspath('derivsleadlag.txt')
    leadlagderivssqmot = abspath('derivssqleadlag.txt')
    
    return(leadlagderivsmot, leadlagderivssqmot)

# create timeseries of high motion volumes for spike regression
def make_spike_reg_matrix(fd,dvars,fd_threshold, dvars_threshold):
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    import numpy as np
    from os.path import abspath

    fd = np.genfromtxt(fd)
    dvars = np.genfromtxt(dvars)
    dvars = (dvars - np.mean(dvars))/np.std(dvars) # convert to standard units
    dvars[0]=0
    spike_ts_mat = ((fd>=fd_threshold) | (dvars>=dvars_threshold)).astype(int)
    n_spikes = sum(spike_ts_mat)
    spike_mat_mat = np.zeros((len(spike_ts_mat),n_spikes))

    y=0
    for x in range(0,len(spike_ts_mat)):
        if spike_ts_mat[x]==1:
            spike_mat_mat[x,y] = 1
            y=y+1

    np.savetxt('spike_matrix.txt',spike_mat_mat)
    np.savetxt('spike_timeseries.txt',spike_ts_mat)

    spike_matrix = abspath('spike_matrix.txt')
    spike_timeseries = abspath('spike_timeseries.txt')
    
    return(spike_matrix, spike_timeseries)

def calc_global_signal(func,mask):
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    import numpy as np
    from os.path import abspath
    from nilearn.masking import apply_mask
    
    func_data = apply_mask(func,mask)
    func_data[func_data==0]=np.nan
    globsig = np.nanmean(func_data,axis=1) 
    np.savetxt('global_signal.txt',globsig)
    global_signal = abspath('global_signal.txt')
    
    return(global_signal)

In [23]:
# get scanner noise
session_noise = Node(Function(input_names=['mask','in_file'], 
                              output_names=['blurred_masked_file'],
                              function=mask_blur_func), name='session_noise')

# get noise associated with WM and CSF
wmcsf_noise = Node(Function(input_names=['mask','in_file'], 
                            output_names=['blurred_masked_file'],
                            function=mask_blur_func), name='wmcsf_noise')

# extract components from session nifti
comp_session_noise = Node(CompCor(repetition_time=TR,
                                  num_components=9,
                                  components_file='components.txt'), name='comp_session_noise')

# extract components from WM-CSF nifti 
comp_wmcsf_noise = Node(CompCor(repetition_time=TR, 
                                num_components=9,
                                components_file='components.txt'), name='comp_wmcsf_noise')

# prepare leadlag motion and derivatives
prep_motion = Node(Function(input_names=['motion_file'], 
                            output_names=['leadlagderivsmot','leadlagderivssqmot'],
                            function=leadlagmatrix), name='prep_motion')

# create the despiking matrix and the censor timeseries files
identify_spikes = Node(Function(input_names=['fd','dvars','fd_threshold', 'dvars_threshold'], 
                                output_names=['spike_matrix','spike_timeseries'],
                                function=make_spike_reg_matrix), name='identify_spikes')
identify_spikes.inputs.fd_threshold=fd_threshold
identify_spikes.inputs.dvars_threshold=dvars_threshold

# calculate global signal regressor
calc_globalsignal = Node(Function(input_names=['func','mask'], 
                                  output_names=['global_signal'], 
                                  function=calc_global_signal),name='calc_globalsignal')

In [ ]:
create_noise_flow = Workflow(name='create_noise_flow')
create_noise_flow.connect([(subinfosource,selectfiles,[('subject_id','subject_id')]),
                           (selectfiles, wmcsf_noise, [('func','in_file')]),
                           (selectfiles, session_noise, [('func','in_file')]),
                           (selectfiles, wmcsf_noise, [('nongm_mask','mask')]),
                           (selectfiles, comp_session_noise, [('brain_mask','mask_files')]),
                           (selectfiles, session_noise, [('nonbrain_mask','mask')]),
                           (selectfiles, comp_wmcsf_noise, [('brain_mask','mask_files')]),
                           
                           (wmcsf_noise, comp_wmcsf_noise, [('blurred_masked_file','realigned_file')]),
                           (session_noise, comp_session_noise, [('blurred_masked_file','realigned_file')]),
                           (wmcsf_noise, datasink, [('blurred_masked_file','wmcsf_noise_file')]),
                           (session_noise, datasink, [('blurred_masked_file','session_noise_file')]),
                           (comp_wmcsf_noise, datasink, [('components_file','subject_wmcsf_comp_noise')]),
                           (comp_session_noise, datasink, [('components_file','subject_session_comp_noise')]),
                           
                           (selectfiles, prep_motion, [('motion','motion_file')]),
                           (selectfiles, identify_spikes,[('fd','fd'),('dvars','dvars')]),
                           (prep_motion, datasink, [('leadlagderivsmot','leadlagderivsmotion'),
                                                    ('leadlagderivssqmot','leadlagderivs_squaremotion')]),
                           (identify_spikes, datasink, [('spike_matrix','motion_spike_matrix'), 
                                                        ('spike_timeseries','motion_spike_timeseries')]),
                           (selectfiles, calc_globalsignal, [('func','func'),('brain_mask','mask')]),
                           (calc_globalsignal,datasink,[('global_signal','global_signal')])
                          ])
create_noise_flow.base_dir = workflow_dir
#create_noise_flow.write_graph(graph2use='flat')
create_noise_flow.run('MultiProc', plugin_args={'n_procs': 6})

210702-08:20:44,630 nipype.workflow INFO:
	 Workflow create_noise_flow settings: ['check', 'execution', 'logging', 'monitoring']
210702-08:20:44,775 nipype.workflow INFO:
	 Running in parallel.
210702-08:20:44,782 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 8 jobs ready. Free memory (GB): 14.40/14.40, Free processors: 6/6.
210702-08:20:44,925 nipype.workflow INFO:
	 [Node] Setting-up "create_noise_flow.selectmask" in "/Users/SEAlab/Documents/PPM/Workflows/create_noise_flow/_subject_id_1196_PP4/_run_num_3/selectmask".210702-08:20:44,926 nipype.workflow INFO:
	 [Node] Setting-up "create_noise_flow.selectmask" in "/Users/SEAlab/Documents/PPM/Workflows/create_noise_flow/_subject_id_1196_PP4/_run_num_2/selectmask".
210702-08:20:44,928 nipype.workflow INFO:
	 [Node] Setting-up "create_noise_flow.selectmask" in "/Users/SEAlab/Documents/PPM/Workflows/create_noise_flow/_subject_id_1196_PP4/_run_num_1/selectmask".

210702-08:20:44,930 nipype.workflow INFO:
	 [Node] Setting-up "creat

	 [Node] Setting-up "create_noise_flow.identify_spikes" in "/Users/SEAlab/Documents/PPM/Workflows/create_noise_flow/_subject_id_1196_PP4/_run_num_3/identify_spikes".

210702-08:20:46,893 nipype.workflow INFO:
	 [Node] Setting-up "create_noise_flow.session_noise" in "/Users/SEAlab/Documents/PPM/Workflows/create_noise_flow/_subject_id_1196_PP4/_run_num_3/session_noise".
210702-08:20:46,893 nipype.workflow INFO:
	 [Node] Setting-up "create_noise_flow.wmcsf_noise" in "/Users/SEAlab/Documents/PPM/Workflows/create_noise_flow/_subject_id_1196_PP4/_run_num_3/wmcsf_noise".210702-08:20:46,893 nipype.workflow INFO:
	 [Node] Setting-up "create_noise_flow.calc_globalsignal" in "/Users/SEAlab/Documents/PPM/Workflows/create_noise_flow/_subject_id_1196_PP4/_run_num_2/calc_globalsignal".


210702-08:20:46,914 nipype.workflow INFO:
	 [Node] Running "calc_globalsignal" ("nipype.interfaces.utility.wrappers.Function")210702-08:20:46,915 nipype.workflow INFO:
	 [Node] Running "identify_spikes" ("nipype.inte

	 [Node] identify_spikes - setting input dvars = /Users/SEAlab/Documents/PPM/fMRIproc/preprocessing/dvars_values/1196_PP4/_run_num_2/out_mot_metric.txt
210702-08:20:48,922 nipype.workflow DEBUG:
	 [Node] Writing pre-exec report to "/Users/SEAlab/Documents/PPM/Workflows/create_noise_flow/_subject_id_1196_PP4/_run_num_2/prep_motion/_report/report.rst"210702-08:20:48,927 nipype.utils DEBUG:
	 Removing contents of /Users/SEAlab/Documents/PPM/Workflows/create_noise_flow/_subject_id_1196_PP4/_run_num_2/identify_spikes

210702-08:20:48,985 nipype.workflow DEBUG:
	 [Node] Writing pre-exec report to "/Users/SEAlab/Documents/PPM/Workflows/create_noise_flow/_subject_id_1196_PP4/_run_num_2/identify_spikes/_report/report.rst"210702-08:20:48,986 nipype.workflow INFO:
	 [Node] Running "prep_motion" ("nipype.interfaces.utility.wrappers.Function")

210702-08:20:48,992 nipype.workflow INFO:
	 [Node] Running "identify_spikes" ("nipype.interfaces.utility.wrappers.Function")
210702-08:20:49,190 nipype.work

	 in_file_/Users/SEAlab/Documents/PPM/fMRIproc/preprocessing/registered_func/1196_PP4/_run_num_2/norm_func.nii.gz

210702-08:20:50,955 nipype.interface DEBUG:
	 mask_file_/Users/SEAlab/Documents/PPM/fMRIproc/preprocessing/wm_csf_mask/1196_PP4/non_gm_mask.nii.gz
210702-08:20:50,958 nipype.interface DEBUG:
	 mask_file_/Users/SEAlab/Documents/PPM/fMRIproc/preprocessing/background_mask/1196_PP4/inverted_whole_brain_mask.nii.gz
210702-08:20:50,960 nipype.interface DEBUG:
	 nan2zeros_True210702-08:20:50,961 nipype.interface DEBUG:
	 nan2zeros_True

210702-08:20:50,963 nipype.interface DEBUG:
	 out_file_masked_file.nii.gz
210702-08:20:50,964 nipype.interface DEBUG:
	 out_file_masked_file.nii.gz
210702-08:20:52,791 nipype.workflow INFO:
	 [MultiProc] Running 6 tasks, and 12 jobs ready. Free memory (GB): 13.20/14.40, Free processors: 0/6.
                     Currently running:
                       * create_noise_flow.wmcsf_noise
                       * create_noise_flow.session_noise
      

## Final Denoising Workflow

In [3]:
#file handling nodes
selectfiles_template={'motion': output_dir + '/motion_parameters/{subject_id}/_run_num_{run_num}/rest_moco.nii.gz.par', 
                      'leadlagderivsmotion': output_dir + '/leadlagderivsmotion/{subject_id}/_run_num_{run_num}/derivsleadlag.txt',
                      'global_signal': output_dir + '/global_signal/{subject_id}/_run_num_{run_num}/global_signal.txt',
                      'func': output_dir + '/registered_func/{subject_id}/_run_num_{run_num}/norm_func.nii.gz',
                      'session': output_dir + '/session_noise_file/{subject_id}/_run_num_{run_num}/blurred_masked_file.nii.gz',
                      'wmcsf': output_dir + '/wmcsf_noise_file/{subject_id}/_run_num_{run_num}/blurred_masked_file.nii.gz',
                      'spike_matrix':output_dir + '/motion_spike_matrix/{subject_id}/_run_num_{run_num}/spike_matrix.txt',
                      'spike_timeseries':output_dir + '/motion_spike_timeseries/{subject_id}/_run_num_{run_num}/spike_timeseries.txt',
                      'mask': output_dir + '/brain_mask/{subject_id}/whole_brain_mask.nii.gz'}
selectfiles=Node(SelectFiles(selectfiles_template),name='selectfiles')
selectfiles.iterables=('run_num', ['0','1','2','3'])

In [7]:
def org_shared_noise(motion, leadlagderivsmotion, spike_matrix, global_signal):
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    from numpy import loadtxt, concatenate,ndim, expand_dims
    from pandas import DataFrame
    from os.path import abspath

    noise_list = []
    for file in [motion, leadlagderivsmotion, spike_matrix]:
        mo = loadtxt(file, dtype=float, comments=None)
        if ndim(mo)<2:
            mo = expand_dims(mo, axis=1)
        noise_list.append(mo)

    shared_noise_data = concatenate(noise_list,axis=1)

    col_names = ['noise_{0}'.format(a) for a in range(0,shared_noise_data.shape[1])] 

    shared_noise = DataFrame(shared_noise_data, columns=col_names)
    shared_noise['global_signal'] = loadtxt(global_signal, dtype=float, comments=None)
    shared_noise.to_csv('shared_noise.csv')
    shared_noise_file = abspath('shared_noise.csv')
    return(shared_noise_file)

def voxelwise_glm(func, shared_noise_file, mask, wmcsf, session):
    from sklearn.preprocessing import MinMaxScaler
    from os.path import abspath
    from numpy import zeros, dot, transpose, sum
    from numpy.linalg import pinv
    from pandas import read_csv, Series
    from nilearn.masking import apply_mask, unmask

    # import data into an array that is timepoints (rows) by voxel number (columns)
    shared_noise = read_csv(shared_noise_file, index_col=0)
    func_data = apply_mask(func, mask)
    #wmcsf_data = apply_mask(wmcsf, mask)
    session_data = apply_mask(session, mask)
    coefficients = zeros((shared_noise.shape[1]+2,func_data.shape[1]))
    resid_data = zeros(func_data.shape)

    # perform voxel-wise matrix inversion
    for x in range(0,func_data.shape[1]):
        #shared_noise['wmcsf'] = wmcsf_data[:,x]
        shared_noise['session'] = session_data[:,x]
        shared_noise['constant'] = 1
        noise_mat = shared_noise.to_numpy()
        y = func_data[:,x]
        inv_mat = pinv(noise_mat)
        coefficients[:,x] = dot(inv_mat,y)
        yhat=sum(transpose(coefficients[:,x])*noise_mat,axis=1)
        resid_data[:,x] = y - transpose(yhat)

    mm = MinMaxScaler(feature_range = (50,150))
    resid_data = mm.fit_transform(resid_data)
    
    resid_image = unmask(resid_data, mask)
    resid_image.to_filename('residuals.nii.gz')

    coeff_image = unmask(coefficients, mask)
    coeff_image.to_filename('weights.nii.gz')
    sample_design_df = shared_noise.to_csv('last_noise_mat.csv')

    weights = abspath('weights.nii.gz')
    sample_design_df = abspath('last_noise_mat.csv')
    residuals = abspath('residuals.nii.gz')

    return(weights,sample_design_df, residuals)

def drop_high_motion_trs(in_file, brain_mask, timeseries_mask):
    from os.path import abspath
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    from nilearn.masking import apply_mask, unmask
    import numpy as np
    
    spikes=np.genfromtxt(timeseries_mask).astype(int)
    vols_to_keep = spikes==0
    func_data = apply_mask(in_file,brain_mask)
    func_data = func_data[vols_to_keep]
    lomo_image = unmask(func_data,brain_mask)
    lomo_image.to_filename('lomo_func.nii.gz')
    out_file = abspath('lomo_func.nii.gz')
    
    return(out_file)

In [8]:
compile_noise_mat = Node(Function(input_names=['motion', 'leadlagderivsmotion', 'spike_matrix','global_signal'],
                                  output_names=['shared_noise_file'],
                                  function=org_shared_noise), 
                         name='compile_noise_mat')

denoise_func = Node(Function(input_names=['func','shared_noise_file','mask','wmcsf','session'], 
                             output_names=['weights','sample_design_df','residuals'],
                             function=voxelwise_glm),
                       name='denoise_func')

# band pass filtering- all rates are in Hz (1/TR)
bandpass = Node(Bandpass(highpass=highpass_freq,
                         lowpass=lowpass_freq, 
                         out_file='resids_bp.nii.gz'), 
                name='bandpass')

drop_himo = Node(Function(input_names=['in_file','brain_mask','timeseries_mask'], 
                          output_names=['out_file'], 
                          function=drop_high_motion_trs), 
                 name='drop_himo')

In [9]:
denoise_flow = Workflow(name='denoise_flow')
denoise_flow.connect([(subinfosource,selectfiles,[('subject_id','subject_id')]),
                      (selectfiles, denoise_func, [('mask','mask'),('func','func'),
                                                   ('wmcsf','wmcsf'),('session','session')]),
                      (selectfiles, compile_noise_mat, [('motion','motion'),
                                                        ('leadlagderivsmotion','leadlagderivsmotion'), 
                                                        ('global_signal','global_signal'),
                                                        ('spike_matrix','spike_matrix')]),
                      (selectfiles, drop_himo, [('mask','brain_mask'),('spike_timeseries','timeseries_mask')]),
                      (compile_noise_mat, denoise_func, [('shared_noise_file','shared_noise_file')]),
                      (denoise_func, bandpass, [('residuals','in_file')]), 
                      (bandpass, drop_himo, [('out_file','in_file')]),
                      
                      (bandpass, datasink, [('out_file','full_proc_func')]),
                      (drop_himo, datasink, [('out_file','lomo_proc_func')]),
                      (denoise_func, datasink, [('weights','denoising_weights'),
                                                ('sample_design_df','denoise_sample_design_df')])
                     ])
denoise_flow.base_dir = workflow_dir
#denoise_flow.write_graph(graph2use='flat')
denoise_flow.run('MultiProc', plugin_args={'n_procs': 4})


210527-10:57:18,598 nipype.workflow INFO:
	 Workflow denoise_flow settings: ['check', 'execution', 'logging', 'monitoring']
210527-10:57:18,818 nipype.workflow INFO:
	 Running in parallel.
210527-10:57:18,824 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 14 jobs ready. Free memory (GB): 14.40/14.40, Free processors: 4/4.
210527-10:57:18,934 nipype.workflow INFO:
	 [Node] Setting-up "denoise_flow.selectfiles" in "/Users/SEAlab/Documents/PPM/Workflows/denoise_flow/_subject_id_1196_Preg5/_run_num_1/selectfiles".
210527-10:57:18,936 nipype.workflow INFO:
	 [Node] Setting-up "denoise_flow.selectfiles" in "/Users/SEAlab/Documents/PPM/Workflows/denoise_flow/_subject_id_1196_Preg5/_run_num_0/selectfiles".210527-10:57:18,938 nipype.workflow INFO:
	 [Node] Setting-up "denoise_flow.selectfiles" in "/Users/SEAlab/Documents/PPM/Workflows/denoise_flow/_subject_id_1196_Preg4/_run_num_1/selectfiles".
210527-10:57:18,939 nipype.workflow INFO:
	 [Node] Setting-up "denoise_flow.selectfiles" in

	 [Node] Running "drop_himo" ("nipype.interfaces.utility.wrappers.Function")
210527-11:16:13,988 nipype.workflow INFO:
	 [MultiProc] Running 4 tasks, and 10 jobs ready. Free memory (GB): 13.60/14.40, Free processors: 0/4.
                     Currently running:
                       * denoise_flow.drop_himo
                       * denoise_flow.bandpass
                       * denoise_flow.denoise_func
                       * denoise_flow.denoise_func
210527-11:16:14,398 nipype.workflow INFO:
	 [Node] Finished "denoise_flow.bandpass".
210527-11:16:15,988 nipype.workflow INFO:
	 [Job 3] Completed (denoise_flow.bandpass).
210527-11:16:16,15 nipype.workflow INFO:
	 [MultiProc] Running 3 tasks, and 11 jobs ready. Free memory (GB): 13.80/14.40, Free processors: 1/4.
                     Currently running:
                       * denoise_flow.drop_himo
                       * denoise_flow.denoise_func
                       * denoise_flow.denoise_func
210527-11:16:16,176 nipype.workflow

	 [Node] Writing post-exec report to "/Users/SEAlab/Documents/PPM/Workflows/denoise_flow/_subject_id_1196_Preg4/_run_num_0/datasink/_report/report.rst"
210527-11:17:03,25 nipype.workflow INFO:
	 [Node] Finished "denoise_flow.datasink".
210527-11:17:04,40 nipype.workflow INFO:
	 [Job 23] Completed (denoise_flow.datasink).
210527-11:17:04,42 nipype.workflow INFO:
	 [MultiProc] Running 3 tasks, and 10 jobs ready. Free memory (GB): 13.80/14.40, Free processors: 1/4.
                     Currently running:
                       * denoise_flow.drop_himo
                       * denoise_flow.denoise_func
                       * denoise_flow.denoise_func
210527-11:17:04,155 nipype.workflow INFO:
	 [Node] Setting-up "denoise_flow.selectfiles" in "/Users/SEAlab/Documents/PPM/Workflows/denoise_flow/_subject_id_1196_Preg3/_run_num_1/selectfiles".
210527-11:17:04,159 nipype.workflow DEBUG:
	 [Node] Hashes: [('force_lists', False), ('raise_on_empty', True), ('run_num', '1'), ('sort_filelist', True

	 sub.str: /Users/SEAlab/Documents/PPM/fMRIproc/preprocessing/denoise_sample_design_df/_subject_id_1196_Preg5/_run_num_1/last_noise_mat.csv -> /Users/SEAlab/Documents/PPM/fMRIproc/preprocessing/denoise_sample_design_df/1196_Preg5/_run_num_1/last_noise_mat.csv using '_subject_id_' -> ''
210527-11:17:08,338 nipype.interface INFO:
	 sub: /Users/SEAlab/Documents/PPM/fMRIproc/preprocessing/denoise_sample_design_df/_subject_id_1196_Preg5/_run_num_1/last_noise_mat.csv -> /Users/SEAlab/Documents/PPM/fMRIproc/preprocessing/denoise_sample_design_df/1196_Preg5/_run_num_1/last_noise_mat.csv210527-11:17:08,338 nipype.workflow INFO:
	 [Node] Running "denoise_func" ("nipype.interfaces.utility.wrappers.Function")

210527-11:17:08,341 nipype.interface DEBUG:
	 copyfile: /Users/SEAlab/Documents/PPM/Workflows/denoise_flow/_subject_id_1196_Preg5/_run_num_1/denoise_func/last_noise_mat.csv /Users/SEAlab/Documents/PPM/fMRIproc/preprocessing/denoise_sample_design_df/1196_Preg5/_run_num_1/last_noise_mat.csv
21

3dBandpass -prefix resids_bp.nii.gz 0.008000 0.090000 /Users/SEAlab/Documents/PPM/Workflows/denoise_flow/_subject_id_1196_Preg5/_run_num_0/bandpass/residuals.nii.gz
210527-11:18:44,132 nipype.workflow INFO:
	 [MultiProc] Running 4 tasks, and 8 jobs ready. Free memory (GB): 13.60/14.40, Free processors: 0/4.
                     Currently running:
                       * denoise_flow.bandpass
                       * denoise_flow.denoise_func
                       * denoise_flow.denoise_func
                       * denoise_flow.denoise_func
210527-11:19:58,450 nipype.workflow INFO:
	 [Node] Finished "denoise_flow.bandpass".
210527-11:20:00,202 nipype.workflow INFO:
	 [Job 9] Completed (denoise_flow.bandpass).
210527-11:20:00,205 nipype.workflow INFO:
	 [MultiProc] Running 3 tasks, and 9 jobs ready. Free memory (GB): 13.80/14.40, Free processors: 1/4.
                     Currently running:
                       * denoise_flow.denoise_func
                       * denoise_flow.denois

...

210527-12:10:21,226 nipype.workflow DEBUG:
	 copying files to wd [execute=True, linksonly=False]
210527-12:10:21,229 nipype.utils DEBUG:
	 /Users/SEAlab/Documents/PPM/Workflows/denoise_flow/_subject_id_1196_PP1/_run_num_0/bandpass/residuals.nii.gz
210527-12:10:21,233 nipype.utils DEBUG:
	 Symlinking File: /Users/SEAlab/Documents/PPM/Workflows/denoise_flow/_subject_id_1196_PP1/_run_num_0/bandpass/residuals.nii.gz->/Users/SEAlab/Documents/PPM/Workflows/denoise_flow/_subject_id_1196_PP1/_run_num_0/denoise_func/residuals.nii.gz
210527-12:10:21,236 nipype.utils DEBUG:
	 /Users/SEAlab/Documents/PPM/Workflows/denoise_flow/_subject_id_1196_PP1/_run_num_0/bandpass/residuals.nii.gz
210527-12:10:21,240 nipype.utils DEBUG:
	 File: /Users/SEAlab/Documents/PPM/Workflows/denoise_flow/_subject_id_1196_PP1/_run_num_0/bandpass/residuals.nii.gz already exists, not overwriting, copy:0
210527-12:10:21,245 nipype.interface DEBUG:
	 highpass_0.008
210527-12:10:21,248 nipype.interface DEBUG:
	 in_file_/Users/

	 sub: /Users/SEAlab/Documents/PPM/fMRIproc/preprocessing/denoise_sample_design_df/_subject_id_1196_PP1/_run_num_1/last_noise_mat.csv -> /Users/SEAlab/Documents/PPM/fMRIproc/preprocessing/denoise_sample_design_df/1196_PP1/_run_num_1/last_noise_mat.csv
210527-12:10:22,185 nipype.interface DEBUG:
	 copyfile: /Users/SEAlab/Documents/PPM/Workflows/denoise_flow/_subject_id_1196_PP1/_run_num_1/denoise_func/last_noise_mat.csv /Users/SEAlab/Documents/PPM/fMRIproc/preprocessing/denoise_sample_design_df/1196_PP1/_run_num_1/last_noise_mat.csv
210527-12:10:22,188 nipype.utils DEBUG:
	 /Users/SEAlab/Documents/PPM/fMRIproc/preprocessing/denoise_sample_design_df/1196_PP1/_run_num_1/last_noise_mat.csv
210527-12:10:22,191 nipype.utils DEBUG:
	 Linking File: /Users/SEAlab/Documents/PPM/fMRIproc/preprocessing/denoise_sample_design_df/1196_PP1/_run_num_1/last_noise_mat.csv->/Users/SEAlab/Documents/PPM/Workflows/denoise_flow/_subject_id_1196_PP1/_run_num_1/denoise_func/last_noise_mat.csv
210527-12:10:22,19

210527-12:11:39,259 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 1 jobs ready. Free memory (GB): 14.40/14.40, Free processors: 4/4.
210527-12:11:39,381 nipype.workflow INFO:
	 [Node] Outdated cache found for "denoise_flow.drop_himo".
210527-12:11:39,383 nipype.workflow INFO:
	 [Node] Setting-up "denoise_flow.drop_himo" in "/Users/SEAlab/Documents/PPM/Workflows/denoise_flow/_subject_id_1196_PP1/_run_num_0/drop_himo".
210527-12:11:39,386 nipype.workflow DEBUG:
	 [Node] Hashes: [('brain_mask', ('/Users/SEAlab/Documents/PPM/fMRIproc/preprocessing/brain_mask/1196_PP1/whole_brain_mask.nii.gz', '47d7a87ba70b4c0e5923b6662dc1a1a0')), ('function_str', "def drop_high_motion_trs(in_file, brain_mask, timeseries_mask):\n    from os.path import abspath\n    from nipype import config, logging\n    config.enable_debug_mode()\n    logging.update_logging(config)\n    from nilearn.masking import apply_mask, unmask\n    import numpy as np\n\n    spikes=np.genfromtxt(timeseries_mask).astype(int)

	 copyfile: /Users/SEAlab/Documents/PPM/Workflows/denoise_flow/_subject_id_1196_PP1/_run_num_0/bandpass/resids_bp.nii.gz /Users/SEAlab/Documents/PPM/fMRIproc/preprocessing/full_proc_func/1196_PP1/_run_num_0/resids_bp.nii.gz
210527-12:12:13,547 nipype.utils DEBUG:
	 /Users/SEAlab/Documents/PPM/fMRIproc/preprocessing/full_proc_func/1196_PP1/_run_num_0/resids_bp.nii.gz
210527-12:12:13,550 nipype.utils DEBUG:
	 Linking File: /Users/SEAlab/Documents/PPM/fMRIproc/preprocessing/full_proc_func/1196_PP1/_run_num_0/resids_bp.nii.gz->/Users/SEAlab/Documents/PPM/Workflows/denoise_flow/_subject_id_1196_PP1/_run_num_0/bandpass/resids_bp.nii.gz
210527-12:12:13,554 nipype.utils DEBUG:
	 /Users/SEAlab/Documents/PPM/fMRIproc/preprocessing/full_proc_func/1196_PP1/_run_num_0/resids_bp.nii.gz
210527-12:12:13,557 nipype.utils DEBUG:
	 File: /Users/SEAlab/Documents/PPM/fMRIproc/preprocessing/full_proc_func/1196_PP1/_run_num_0/resids_bp.nii.gz already exists, not overwriting, copy:1
210527-12:12:13,565 nipype

RuntimeError: Workflow did not execute cleanly. Check log for details

In [3]:
# merge files according to acquisition condition (fixation/rest or mlp/movie)
from glob import glob
from os import mkdir
from nipype.interfaces.fsl import Merge

merge = Merge()
merge.inputs.dimension='t'
for sub in subject_ids: 
    files = glob(output_dir + '/lomo_proc_func/_{0}/*/lomo_func.nii.gz'.format(sub))
    merge.inputs.in_files = files
    merge.inputs.merged_file = output_dir + '/lomo_proc_func/_{0}/merged_func.nii.gz'.format(sub)
    merge.run()        

In [16]:
import nibabel as nib
from sklearn.preprocessing import StandardScaler
from nilearn.masking import unmask, apply_mask

ss - StandardScaler(0,100)
for sub in subject_ids:
    merged_file = output_dir + '/lomo_proc_func/_{0}/merged_func.nii.gz'.format(sub)
    mask = output_dir + '/brain_mask_dilated/{0}/whole_brain_D1_mask.nii.gz'.format(sub)
    func_data = apply_mask(merged_file, mask)
    func_data = ss.fit_transform(func_data)
    img = unmask(func_data, mask)
    nib.save(output_dir + '/lomo_proc_func/_{0}/merged_func_ss.nii.gz'.format(sub))
    

NameError: name 'ss' is not defined